!pip install pydub

In [3]:
import pandas as pd

def display_unique_emotion_labels(file_path, column_name):
    # Read the dataset
    df = pd.read_csv(file_path)
    
    # Ensure the column exists
    if column_name not in df.columns:
        raise ValueError(f"Column '{column_name}' not found in the dataset.")
    
    # Get unique labels in the specified column
    unique_labels = df[column_name].unique()
    
    # Display the unique labels
    print(f"Unique labels in the '{column_name}' column (case-sensitive):")
    for label in unique_labels:
        print(label)


file_path = '/kaggle/input/final-dataset-csv-wav-minor/final_csv_formatted_merged_file_preprocessed.csv'  # Update with the actual file path
column_name = 'Emotion'  # Update with the actual column name
display_unique_emotion_labels(file_path, column_name)


Unique labels in the 'Emotion' column (case-sensitive):
HAPPY
SAD
NEUTRAL
ANGER
SURPRISED
FEAR
DISGUST


In [4]:
import pandas as pd
import os

def preprocess_emotion_labels(input_file_path, output_file_path, column_name):
    # Read the dataset
    df = pd.read_csv(input_file_path)
    
    # Ensure the column exists
    if column_name not in df.columns:
        raise ValueError(f"Column '{column_name}' not found in the dataset.")
    
    # Define the label mappings
    label_mappings = {
        'happiness': 'HAPPY',
        'Happy/Joy': 'HAPPY',
        'happy': 'HAPPY',
        'sad': 'SAD',
        'Sad': 'SAD',
        'sadness': 'SAD',
        'calm': 'NEUTRAL',
        'neutral': 'NEUTRAL',
        'Neutral': 'NEUTRAL',
        'boredom': 'NEUTRAL',
        'angry': 'ANGER',
        'Anger': 'ANGER',
        'anger': 'ANGER',
        'surprised': 'SURPRISED',
        'fearful': 'FEAR',
        'anxiety/fear': 'FEAR',
        'Fear': 'FEAR',
        'fear': 'FEAR',
        'disgust': 'DISGUST',
        'Disgust': 'DISGUST'
    }
    
    # Apply the label mappings to the specified column
    df[column_name] = df[column_name].map(label_mappings).fillna(df[column_name])
    
    # Save the preprocessed dataset to a new file
    df.to_csv(output_file_path, index=False)
    print(f"Preprocessed file saved as: {output_file_path}")


input_file_path = '/kaggle/input/final-dataset-csv-wav-minor/final_csv_formatted_merged_file_preprocessed.csv'  # Update with the actual input file path
output_file_path = '/kaggle/working/final_csv_formatted_merged_file_preprocessed.csv'  # Update with the desired output file path
column_name = 'Emotion'  # Update with the actual column name
preprocess_emotion_labels(input_file_path, output_file_path, column_name)


Preprocessed file saved as: /kaggle/working/final_csv_formatted_merged_file_preprocessed.csv


In [5]:
import pandas as pd

csv_file = '/kaggle/input/final-dataset-csv-wav-minor/final_csv_formatted_merged_file_preprocessed.csv'  # Path to your updated CSV file
data = pd.read_csv(csv_file)


In [6]:
data

,File Name,Emotion,Gender
0,03-01-03-01-02-02-13.wav,HAPPY,MALE
1,03-02-04-01-01-01-10.wav,SAD,FEMALE
2,03-02-01-01-02-02-24.wav,NEUTRAL,FEMALE
3,03-02-05-02-01-01-02.wav,ANGER,FEMALE
4,03-01-08-01-01-01-02.wav,SURPRISED,FEMALE
...,...,...,...
10424,1091_WSI_DIS_XX.wav,DISGUST,FEMALE
10425,1091_WSI_FEA_XX.wav,FEAR,FEMALE
10426,1091_WSI_HAP_XX.wav,HAPPY,FEMALE
10427,1091_WSI_NEU_XX.wav,NEUTRAL,FEMALE


In [7]:
import librosa
import numpy as np

max_pad_len = 174  # Adjust based on your dataset

def extract_features(file_path):
    audio, sr = librosa.load(file_path, sr=None)
    mfccs = librosa.feature.mfcc(audio, sr=sr, n_mfcc=40)
    pad_width = max_pad_len - mfccs.shape[1]
    mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')
    return mfccs


In [8]:
import os
import csv
import numpy as np
import librosa

# Function to load audio data from a WAV file
def load_audio(file_path, sr=22050):
    
    try:
        # Load audio data from the WAV file
        audio_data, _ = librosa.load(file_path, sr=sr)
        return audio_data
    except Exception as e:
        print(f"Error loading audio file: {file_path}. Error message: {e}")
        return None

# Path to the directory containing the WAV files
data_dir = '/kaggle/input/final-dataset-csv-wav-minor/AUDIO_FILES/AUDIO_FILES'

# Path to the CSV file containing labels
csv_file_path = '/kaggle/input/final-dataset-csv-wav-minor/final_csv_formatted_merged_file_preprocessed.csv'

# List to store extracted features and corresponding labels
X = []
labels = []

# Parameters for fixed-length MFCC features
max_mfcc_length = 100  
n_mfcc = 13

# Read labels from the CSV file
label_dict = {}
with open(csv_file_path, 'r') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  
    for row in reader:
        file_name, gender_label, emotion_label = row
        label_dict[file_name] = (emotion_label, gender_label)

# Iterate over WAV files in the directory
for file_name in os.listdir(data_dir):
    if file_name.endswith('.wav'):
        file_path = os.path.join(data_dir, file_name)  # Full path to the WAV file

        # Load the audio file
        audio_data = load_audio(file_path)

        if audio_data is not None:
            
            mfcc_features = librosa.feature.mfcc(y=audio_data, sr=22050, n_mfcc=n_mfcc, n_fft=1024, hop_length=512)

            
            if mfcc_features.shape[1] < max_mfcc_length:
                
                mfcc_features = np.pad(mfcc_features, ((0, 0), (0, max_mfcc_length - mfcc_features.shape[1])), mode='constant')
            elif mfcc_features.shape[1] > max_mfcc_length:
                
                mfcc_features = mfcc_features[:, :max_mfcc_length]

            # Append MFCC features
            X.append(mfcc_features)

            
            if file_name in label_dict:
                emotion_label, gender_label = label_dict[file_name]
                
                labels.append((emotion_label, gender_label))
            else:
                print(f"Label not found for file: {file_name}")
        else:
            print(f"Failed to load audio file: {file_path}")


X = np.array(X)
labels = np.array(labels)

print("MFCC features shape:", X.shape)
print("Labels shape:", labels.shape)


MFCC features shape: (10429, 13, 100)
Labels shape: (10429, 2)


In [9]:
from sklearn.model_selection import train_test_split

labels_reshaped = np.array(labels)  
labels_reshaped = labels_reshaped.reshape(-1, 2)  


X_train, X_test, y_train, y_test = train_test_split(X, labels_reshaped, test_size=0.2, random_state=42)

In [10]:
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical



y_train_emotion = np.array(y_train[:, 0])
y_train_gender = np.array(y_train[:, 1])

y_test_emotion = np.array(y_test[:, 0])
y_test_gender = np.array(y_test[:, 1])


emotion_encoder = LabelEncoder()
gender_encoder = LabelEncoder()

y_train_emotion_encoded = emotion_encoder.fit_transform(y_train_emotion)
y_test_emotion_encoded = emotion_encoder.transform(y_test_emotion)

y_train_gender_encoded = gender_encoder.fit_transform(y_train_gender)
y_test_gender_encoded = gender_encoder.transform(y_test_gender)

y_train_emotion_one_hot = to_categorical(y_train_emotion_encoded)
y_test_emotion_one_hot = to_categorical(y_test_emotion_encoded)

y_train_gender_one_hot = to_categorical(y_train_gender_encoded)
y_test_gender_one_hot = to_categorical(y_test_gender_encoded)

X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], X_train.shape[2], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], X_test.shape[2], 1))


In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

datagen.fit(X_train)

In [12]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D, MaxPooling2D, Dropout, LSTM, TimeDistributed, BatchNormalization

input_shape = (X_train.shape[1], X_train.shape[2], 1)

inputs = Input(shape=input_shape)


conv_1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
conv_1 = BatchNormalization()(conv_1)
conv_1 = MaxPooling2D((2, 2))(conv_1)

conv_2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv_1)
conv_2 = BatchNormalization()(conv_2)
conv_2 = MaxPooling2D((2, 2))(conv_2)

conv_3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv_2)
conv_3 = BatchNormalization()(conv_3)
conv_3 = MaxPooling2D((2, 2))(conv_3)

shape = conv_3.shape
conv_3 = TimeDistributed(Flatten())(conv_3)

lstm_1 = LSTM(128, return_sequences=False)(conv_3)

dense_1 = Dense(64, activation='relu')(lstm_1)
dense_1 = Dropout(0.5)(dense_1)

emotion_output = Dense(y_train_emotion_one_hot.shape[1], activation='softmax', name='emotion_output')(dense_1)
gender_output = Dense(y_train_gender_one_hot.shape[1], activation='softmax', name='gender_output')(dense_1)

model = Model(inputs=inputs, outputs=[emotion_output, gender_output])

model.compile(optimizer='adam', 
              loss={'emotion_output': 'categorical_crossentropy', 'gender_output': 'categorical_crossentropy'},
              metrics={'emotion_output': 'accuracy', 'gender_output': 'accuracy'})

model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 13, 100,   │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 13, 100,   │        320 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 13, 100,   │        128 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 6, 50, 32) │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 6, 50, 64) │     18,496 │ max_pooling2d[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 6, 50, 64) │        256 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 3, 25, 64) │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 3, 25,     │     73,856 │ max_pooling2d_1[… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 3, 25,     │        512 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_2     │ (None, 1, 12,     │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed    │ (None, 1, 1536)   │          0 │ max_pooling2d_2[… │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ (None, 128)       │    852,480 │ time_distributed… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 64)        │      8,256 │ lstm[0][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 64)        │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ emotion_output      │ (None, 2)         │        130 │ dropout[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gender_output       │ (None, 7)         │        455 │ dropout[0][0]     │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 954,889 (3.64 MB)

 Trainable params: 954,441 (3.64 MB)

 Non-trainable params: 448 (1.75 KB)

In [13]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D, MaxPooling2D, Dropout, LSTM, TimeDistributed, BatchNormalization

input_shape = (X_train.shape[1], X_train.shape[2], 1)
inputs = Input(shape=input_shape)

# Convolutional Layers
conv_1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
conv_1 = BatchNormalization()(conv_1)
conv_1 = MaxPooling2D((2, 2), padding='same')(conv_1)

conv_2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv_1)
conv_2 = BatchNormalization()(conv_2)
conv_2 = MaxPooling2D((2, 2), padding='same')(conv_2)

conv_3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv_2)
conv_3 = BatchNormalization()(conv_3)
conv_3 = MaxPooling2D((2, 2), padding='same')(conv_3)

conv_4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv_3)
conv_4 = BatchNormalization()(conv_4)
conv_4 = MaxPooling2D((2, 2), padding='same')(conv_4)

# Flatten and LSTM
shape = conv_4.shape
conv_4 = TimeDistributed(Flatten())(conv_4)
lstm_1 = LSTM(128, return_sequences=False)(conv_4)

# Dense Layers
dense_1 = Dense(128, activation='relu')(lstm_1)
dense_1 = Dropout(0.5)(dense_1)
dense_2 = Dense(64, activation='relu')(dense_1)
dense_2 = Dropout(0.5)(dense_2)

# Output Layers
emotion_output = Dense(y_train_emotion_one_hot.shape[1], activation='softmax', name='emotion_output')(dense_2)
gender_output = Dense(y_train_gender_one_hot.shape[1], activation='softmax', name='gender_output')(dense_2)

# Model Compilation
model = Model(inputs=inputs, outputs=[emotion_output, gender_output])
model.compile(optimizer='adam', 
              loss={'emotion_output': 'categorical_crossentropy', 'gender_output': 'categorical_crossentropy'},
              metrics={'emotion_output': 'accuracy', 'gender_output': 'accuracy'})

model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 13, 100,   │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 13, 100,   │        320 │ input_layer_1[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 13, 100,   │        128 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_3     │ (None, 7, 50, 32) │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 7, 50, 64) │     18,496 │ max_pooling2d_3[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 7, 50, 64) │        256 │ conv2d_4[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_4     │ (None, 4, 25, 64) │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 4, 25,     │     73,856 │ max_pooling2d_4[… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 4, 25,     │        512 │ conv2d_5[0][0]    │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_5     │ (None, 2, 13,     │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_6 (Conv2D)   │ (None, 2, 13,     │    295,168 │ max_pooling2d_5[… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 2, 13,     │      1,024 │ conv2d_6[0][0]    │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_6     │ (None, 1, 7, 256) │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed_1  │ (None, 1, 1792)   │          0 │ max_pooling2d_6[… │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ (None, 128)       │    983,552 │ time_distributed… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 128)       │     16,512 │ lstm_1[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 128)       │          0 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 64)        │      8,256 │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 1,398,665 (5.34 MB)

 Trainable params: 1,397,705 (5.33 MB)

 Non-trainable params: 960 (3.75 KB)

In [14]:
from tensorflow.keras.optimizers import Adam

# Compile the model with a lower learning rate
model.compile(optimizer=Adam(learning_rate=0.0001), 
              loss={'emotion_output': 'categorical_crossentropy', 'gender_output': 'categorical_crossentropy'},
              metrics={'emotion_output': 'accuracy', 'gender_output': 'accuracy'})

In [16]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Define early stopping and model checkpoint
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('/kaggle/working/best_model.keras', monitor='val_loss', save_best_only=True)

# Train the model
history = model.fit(X_train, 
                    {'emotion_output': y_train_emotion_one_hot, 'gender_output': y_train_gender_one_hot}, 
                    epochs=100, 
                    batch_size=32, 
                    validation_data=(X_test, {'emotion_output': y_test_emotion_one_hot, 'gender_output': y_test_gender_one_hot}),
                    callbacks=[early_stopping, model_checkpoint])

# Evaluate the model
model.evaluate(X_test, {'emotion_output': y_test_emotion_one_hot, 'gender_output': y_test_gender_one_hot})


Epoch 1/100
261/261 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - emotion_output_accuracy: 0.9669 - gender_output_accuracy: 0.5463 - loss: 1.3130 - val_emotion_output_accuracy: 0.9516 - val_gender_output_accuracy: 0.5211 - val_loss: 1.4218
Epoch 2/100
261/261 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - emotion_output_accuracy: 0.9747 - gender_output_accuracy: 0.5576 - loss: 1.2420 - val_emotion_output_accuracy: 0.9473 - val_gender_output_accuracy: 0.4976 - val_loss: 1.4590
Epoch 3/100
261/261 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - emotion_output_accuracy: 0.9772 - gender_output_accuracy: 0.5864 - loss: 1.1814 - val_emotion_output_accuracy: 0.9497 - val_gender_output_accuracy: 0.5177 - val_loss: 1.4264
Epoch 4/100
261/261 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - emotion_output_accuracy: 0.9734 - gender_output_accuracy: 0.6052 - loss: 1.1526 - val_emotion_output_accuracy: 0.9549 - val_gender_output_accuracy: 0.5077 - val_loss: 1.4278
Epoch 5/100
261/261 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - emotion_output_accuracy:

[1.4217921495437622, 0.9515819549560547, 0.5210930109024048]

In [ ]:
# Train the model
history = model.fit(X_train, 
                    {'emotion_output': y_train_emotion_one_hot, 'gender_output': y_train_gender_one_hot}, 
                    epochs=50, 
                    batch_size=32, 
                    validation_data=(X_test, {'emotion_output': y_test_emotion_one_hot, 'gender_output': y_test_gender_one_hot}))

# Evaluate the model
model.evaluate(X_test, {'emotion_output': y_test_emotion_one_hot, 'gender_output': y_test_gender_one_hot})


In [37]:
# Evaluate the model
evaluation_results = model.evaluate(X_test, 
                                    {'emotion_output': y_test_emotion_one_hot, 'gender_output': y_test_gender_one_hot})

66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - emotion_output_accuracy: 0.9637 - gender_output_accuracy: 0.5720 - loss: 2.6244


In [42]:
# Assuming evaluation_results is a list obtained from model.evaluate()
print(f"Total Loss: {evaluation_results[0]}")
print(f"Gender Classification Accuracy: {evaluation_results[1] * 100:.2f}%")
print(f"Emotion Classification Accuracy: {evaluation_results[2] * 100:.2f}%")


Total Loss: 2.605487585067749
Gender Classification Accuracy: 96.84%
Emotion Classification Accuracy: 56.28%


In [25]:
model.save('speech_emotion_gender_recognition_crnn_emodb.h5')


In [57]:
import librosa
import numpy as np
import tensorflow as tf

# Function to preprocess audio file and extract MFCC features
def preprocess_audio(audio_file, n_mfcc=13, max_mfcc_length=100):
    # Load audio file
    y, sr = librosa.load(audio_file, sr=None)
    
    # Extract MFCC features
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    
    # Pad or truncate MFCCs to fixed length
    if mfccs.shape[1] < max_mfcc_length:
        mfccs = np.pad(mfccs, ((0, 0), (0, max_mfcc_length - mfccs.shape[1])), mode='constant')
    else:
        mfccs = mfccs[:, :max_mfcc_length]
    
    return mfccs

# Path to the saved model
model_path = "/kaggle/working/speech_emotion_gender_recognition_crnn_emodb.h5"

# Load the saved model
loaded_model = tf.keras.models.load_model(model_path)

# Path to the audio file you want to predict
audio_file_path = "/kaggle/input/final-dataset-csv-wav-minor/AUDIO_FILES/AUDIO_FILES/03-01-01-01-01-01-02.wav"

# Preprocess the audio file
mfcc_features = preprocess_audio(audio_file_path)

# Reshape MFCC features to match the input shape expected by the model
mfcc_features = mfcc_features.reshape(1, 13, 100, 1)

# Make predictions using the loaded model
emotion_prediction, gender_prediction = loaded_model.predict(mfcc_features)

# Decode the predicted labels
predicted_emotion_label = emotion_encoder.inverse_transform(np.argmax(emotion_prediction, axis=1))
predicted_gender_label = gender_encoder.inverse_transform(np.argmax(gender_prediction, axis=1))

print("Predicted Gender:", predicted_emotion_label[0])
print("Predicted Emotion:", predicted_gender_label[0])


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 278ms/step
Predicted Gender: FEMALE
Predicted Emotion: NEUTRAL


In [58]:
import numpy as np

# Make predictions
emotion_predictions, gender_predictions = model.predict(X_test)

# Get the predicted classes
predicted_emotion_labels = np.argmax(emotion_predictions, axis=1)
predicted_gender_labels = np.argmax(gender_predictions, axis=1)

# Get the true labels
true_emotion_labels = np.argmax(y_test_emotion_one_hot, axis=1)
true_gender_labels = np.argmax(y_test_gender_one_hot, axis=1)

# Calculate the combined accuracy
correct_predictions = np.sum((predicted_emotion_labels == true_emotion_labels) & 
                             (predicted_gender_labels == true_gender_labels))
combined_accuracy = correct_predictions / len(X_test)

print(f'Combined Accuracy: {combined_accuracy * 100:.2f}%')


66/66 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
Combined Accuracy: 54.79%


In [24]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D, MaxPooling2D, Dropout, BatchNormalization, Bidirectional, LSTM, Reshape
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

input_shape = (X_train.shape[1], X_train.shape[2], 1)

inputs = Input(shape=input_shape)

# Convolutional Layers
conv_1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
conv_1 = BatchNormalization()(conv_1)
conv_1 = MaxPooling2D((2, 2), padding='same')(conv_1)

conv_2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv_1)
conv_2 = BatchNormalization()(conv_2)
conv_2 = MaxPooling2D((2, 2), padding='same')(conv_2)

conv_3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv_2)
conv_3 = BatchNormalization()(conv_3)
conv_3 = MaxPooling2D((2, 2), padding='same')(conv_3)

conv_4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv_3)
conv_4 = BatchNormalization()(conv_4)
conv_4 = MaxPooling2D((2, 2), padding='same')(conv_4)

conv_5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv_4)
conv_5 = BatchNormalization()(conv_5)
conv_5 = MaxPooling2D((2, 2), padding='same')(conv_5)

# Flatten the output of the convolutional layers
conv_flattened = Flatten()(conv_5)

# Reshape to fit LSTM input shape
timesteps = conv_5.shape[1]
features = conv_5.shape[2] * conv_5.shape[3]  # Combine last two dimensions
reshaped_conv = Reshape((timesteps, features))(conv_flattened)

# Bidirectional LSTM Layer
lstm_1 = Bidirectional(LSTM(128, return_sequences=False))(reshaped_conv)

# Fully Connected Layer
dense_1 = Dense(64, activation='relu')(lstm_1)
dense_1 = Dropout(0.5)(dense_1)

# Output Layers
emotion_output = Dense(y_train_emotion_one_hot.shape[1], activation='softmax', name='emotion_output')(dense_1)
gender_output = Dense(y_train_gender_one_hot.shape[1], activation='softmax', name='gender_output')(dense_1)

model = Model(inputs=inputs, outputs=[emotion_output, gender_output])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), 
              loss={'emotion_output': 'categorical_crossentropy', 'gender_output': 'categorical_crossentropy'},
              metrics={'emotion_output': 'accuracy', 'gender_output': 'accuracy'})

model.summary()



early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.keras', monitor='val_accuracy', save_best_only=True)

# Train the model
history = model.fit(X_train, 
                    {'emotion_output': y_train_emotion_one_hot, 'gender_output': y_train_gender_one_hot}, 
                    epochs=100, 
                    batch_size=32, 
                    validation_data=(X_test, {'emotion_output': y_test_emotion_one_hot, 'gender_output': y_test_gender_one_hot}),
                    callbacks=[early_stopping, model_checkpoint])

# Evaluate the model
model.evaluate(X_test, {'emotion_output': y_test_emotion_one_hot, 'gender_output': y_test_gender_one_hot})


Model: "functional_19"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_9       │ (None, 13, 100,   │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_37 (Conv2D)  │ (None, 13, 100,   │        320 │ input_layer_9[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 13, 100,   │        128 │ conv2d_37[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_37    │ (None, 7, 50, 32) │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_38 (Conv2D)  │ (None, 7, 50, 64) │     18,496 │ max_pooling2d_37… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 7, 50, 64) │        256 │ conv2d_38[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_38    │ (None, 4, 25, 64) │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_39 (Conv2D)  │ (None, 4, 25,     │     73,856 │ max_pooling2d_38… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 4, 25,     │        512 │ conv2d_39[0][0]   │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_39    │ (None, 2, 13,     │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_40 (Conv2D)  │ (None, 2, 13,     │    295,168 │ max_pooling2d_39… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 2, 13,     │      1,024 │ conv2d_40[0][0]   │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_40    │ (None, 1, 7, 256) │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_41 (Conv2D)  │ (None, 1, 7, 512) │  1,180,160 │ max_pooling2d_40… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 1, 7, 512) │      2,048 │ conv2d_41[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_41    │ (None, 1, 4, 512) │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_9 (Flatten) │ (None, 2048)      │          0 │ max_pooling2d_41… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_7 (Reshape) │ (None, 1, 2048)   │          0 │ flatten_9[0][0] 

 Total params: 3,818,249 (14.57 MB)

 Trainable params: 3,816,265 (14.56 MB)

 Non-trainable params: 1,984 (7.75 KB)

Epoch 1/100
261/261 ━━━━━━━━━━━━━━━━━━━━ 11s 19ms/step - emotion_output_accuracy: 0.6528 - gender_output_accuracy: 0.2737 - loss: 2.3797 - val_emotion_output_accuracy: 0.7963 - val_gender_output_accuracy: 0.3931 - val_loss: 2.0732
Epoch 2/100
  9/261 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - emotion_output_accuracy: 0.7632 - gender_output_accuracy: 0.4022 - loss: 1.9943

/opt/conda/lib/python3.10/site-packages/keras/src/callbacks/model_checkpoint.py:206: UserWarning: Can save best model only with val_accuracy available, skipping.
  self._save_model(epoch=epoch, batch=None, logs=logs)


261/261 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - emotion_output_accuracy: 0.8542 - gender_output_accuracy: 0.3873 - loss: 1.9026 - val_emotion_output_accuracy: 0.8840 - val_gender_output_accuracy: 0.4415 - val_loss: 1.7232
Epoch 3/100
261/261 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - emotion_output_accuracy: 0.9065 - gender_output_accuracy: 0.4376 - loss: 1.6836 - val_emotion_output_accuracy: 0.8897 - val_gender_output_accuracy: 0.4851 - val_loss: 1.6114
Epoch 4/100
261/261 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - emotion_output_accuracy: 0.9359 - gender_output_accuracy: 0.4856 - loss: 1.5177 - val_emotion_output_accuracy: 0.9223 - val_gender_output_accuracy: 0.5230 - val_loss: 1.4674
Epoch 5/100
261/261 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - emotion_output_accuracy: 0.9460 - gender_output_accuracy: 0.5217 - loss: 1.3998 - val_emotion_output_accuracy: 0.9415 - val_gender_output_accuracy: 0.5034 - val_loss: 1.4448
Epoch 6/100
261/261 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - emotion_output_accuracy: 0.9622 - ge

[1.3616548776626587, 0.9304890036582947, 0.5455417037010193]